In [12]:
import math
import random
import time
from graph.graph import read_graph_from_csv

In [13]:
graph_file = '../graphs/dash/dash.csv'

In [14]:
m = 10 # number of tasks
n = 100 # number of candidate workers

initialize workers by their ids

In [15]:
from stable_matching.TaskWorker import Worker

g = read_graph_from_csv(graph_file,0)
workers = []
worker_ids = random.sample(list(g.nodes.keys()),n)
for i in range(n):
    workers.append(Worker(idx = worker_ids[i]))
del g

initialize tasks : (ids, budget, reverse reachable set, Q)

In [16]:
from graph.QIM import sampling
from utils.funcs import max_k
from stable_matching.TaskWorker import Task
import scipy.stats as stats

mu = 1.0
sigma = 0.1

tasks = []
graph_ids = random.sample(range(100),m)

# allocate budget to every task, sum of all budget is total_budget
avg_budget  = 10
total_budget = avg_budget * m
points = [0]+sorted(random.sample(range(1,total_budget),m-1))+[total_budget]
budgets = [points[i+1]-points[i] for i in range(len(points)-1)]

for i in range(m):
    G = read_graph_from_csv(graph_file,graph_ids[i])
    
    # generate cost of all candidate worker, task cost is generate from a trunc gaussian distribution mu = 1.0, sigma = 0.1, lb = 0.5, ub = 1.5
    X = stats.truncnorm((0.5-mu)/sigma, (1.5-mu)/sigma, loc= mu, scale= sigma).rvs(n)
    costs = {}
    for j in range(n):
        costs[workers[j]] = X[j]
        
    # values of all workers
    values = {}
    Q  = 0.0
    for v in G.nodes:
        values[v] = random.uniform(0.0, 1.0)
        Q += values[v]
        
    budget = budgets[i]
    print('budget:{}'.format(budget))
    start = time.time()
    # generate hyper graph of reverse reachable set in graph G
    k = max_k(budget, costs)
    print('k:{}'.format(k))
    RR = sampling(graph=G, C=worker_ids, k=k, delta=1/n, epsilon=0.01, values=values)
    print('time to generate RR:{}'.format(time.time()-start))
    
    #initialize tasks
    tasks.append(Task(idx=i, budget=budget, R=RR, Q=Q))
    tasks[i].initialize(costs)
    tasks[i].set_choice_matroid(D = k )

budget:11
k:13
i_max:34
time to generate RR:1.106518030166626
budget:14
k:16
i_max:34
time to generate RR:1.049278736114502
budget:2
k:2
i_max:37
time to generate RR:4.038794994354248
budget:20
k:22
i_max:34
time to generate RR:0.7289378643035889
budget:12
k:14
i_max:34
time to generate RR:0.8976690769195557
budget:6
k:7
i_max:35
time to generate RR:1.4221882820129395
budget:12
k:14
i_max:34
time to generate RR:0.9048364162445068
budget:10
k:11
i_max:35
time to generate RR:0.8177247047424316
budget:11
k:12
i_max:35
time to generate RR:1.0120563507080078
budget:2
k:2
i_max:37
time to generate RR:8.01326870918274


In [17]:
for worker in workers:
    value_dict = {}
    for t in tasks:
        value_dict[t] = random.random()
    worker.set_preference(value_dict)

In [18]:
from stable_matching.stableMatching import generalized_da
for t in tasks:
    t.refresh()
for w in workers:
    w.refresh()
generalized_da(tasks,workers)

In [19]:
result_dict = {}

In [20]:
from stable_matching.TaskWorker import outward_satisfactory
result_dict['outward_satisfactory'] = outward_satisfactory(tasks,workers)

In [21]:
from stable_matching.TaskWorker import individual_rationality_tasks
result_dict['individual_rationality'] = individual_rationality_tasks(tasks)

In [22]:
from stable_matching.TaskWorker import gamma_workers
Sum = 0
Max = 0
Min = 1000
for task in tasks:
    q =  task.Q * gamma_workers(task.R,task.students())/len(task.R)/task.budget 
    Sum += q
    if q> Max:
        Max = q
    if q< Min:
        Min = q
result_dict['average_density'] = Sum/len(tasks)
result_dict['maximum_density'] = Max
result_dict['minimum_density'] = Min

In [23]:
result_dict

{'outward_satisfactory': -3.108108108108108,
 'individual_rationality': [[11, 4.232877250438186],
  [14, 3.826517502531262],
  [2, 0.9458061418363184],
  [20, 5.804356269088864],
  [12, 4.702030070970412],
  [6, 2.7601237260154825],
  [12, 3.918067451776059],
  [10, 4.039263490728995],
  [11, 4.812120169503403],
  [2, 1.0824574266002218]],
 'average_density': 1.7224400652891922,
 'maximum_density': 3.25761728667552,
 'minimum_density': 0.9066163421495945}